In [73]:
import ete3
from ete3 import Tree
import os
import os
import glob
import numpy as np
from Bio import AlignIO 
import random
import re
import copy

In [74]:
t = Tree("(((A:0.687, B:0.601):1.244,(C:0.804,(D:0.015, E:0.017):0.79):0.333):0.555,F:1.001);")

In [76]:
os.getcwd()
#os.chdir("./Alignments/block_aln/")

'/home/julybel/trash/sas/Alignments/block_aln'

# Getting block-alignment

In [ ]:
for file in glob.glob("*.fasta"):
    with open(file, 'r') as fname:
        matrix = np.array([np.array(rec.seq) for rec in AlignIO.read(fname, "fasta")])
        names = [rec.name for rec in AlignIO.read("PF00004_1.fasta", "fasta")]
        indexes = [num for num in range(matrix.shape[1]) if list(matrix[:,num]).count("-") > len(matrix[:,num]) * 0.5]
        matrix = np.delete(matrix, indexes, axis=1)
    with open("./block_aln/"+file.split(".")[0]+"_block.fasta", 'w') as fname:
        for name, seq in zip(names, list(matrix)):
            fname.write(">" + name + '\n' + ''.join(list(seq)) + '\n')

# New way

In [132]:
class nd:
    def __init__(self, seq):
        self.seq = seq
    def children(self, children):
        self.children = children
    def dist(self, dist):
        self.dist = dist
    def name(self, name):
        self.name = name

In [166]:
class parser(nd):
    def __init__(self, x, tree):
        self.x = x
        self.t = tree
        super().__init__(0)

    #Choose random alignment (with seq_len more than self.x)
    def random_aln(self):
        while True:
            file = random.choice([file for file in glob.glob("*.fasta")])
            aln = AlignIO.read(open(file, 'r'), "fasta")
            if len(str(aln[0].seq)) >= self.x:
                break
        self.file = file
        self.alignment_dict = {record.id:str(record.seq) for record in aln}
        
    #Make root sequence and pin columns   
    def root_seq(self):
        column_id = [random.randint(0, self.x) for _ in range(self.x)]
        a = AlignIO.read(open(self.file, 'r'), "fasta")
        names, seq = [],''
        for num in range(self.x):
            record = random.randint(0, len(a)-1)
            names.append(a[record].id)
            seq += str(a[record].seq)[column_id[num]] 
        self.columns = column_id
        self.seq = seq
        self.names = names
             
    #Make all_dists and all_id lists
    def dists(self):
        all_id, all_dists, values, one_id = [], [], [], ''
        with open(self.file[:-12]+".dist", 'r') as fname:
            for string in fname:
                if re.search("[A-Za-z]+", string) != None:
                    all_id.append(one_id)
                    all_dists.append(values)
                    one_id, values = string.split()[0], string.split()[1:]
                else:
                    values += string.split()
            all_id.append(one_id)
            all_dists.append(values)
        self.all_id = all_id[1:]
        self.all_dists = all_dists[1:]
        
    #NORMALIZATION
    def normal(self):
        container = []
        tree_max = 1.244
        for subset in self.all_dists:
            dist_max = max([float(num) for num in subset])
            dist_min = 10
            for i in subset:
                if float(i) < float(dist_min) and float(i) != float(0):
                    dist_min = i
            dist_diff = float(dist_max) - float(dist_min)
            alpha = dist_diff / (1.244 - 0.015)
            container.append(list(map(lambda x: str(float(x)*alpha), subset)))
        #print(container)
        self.all_dists = container
            
    #Make sequences
    def make_sample(self):
        nodes, graph, lvl = [], [], -1
        
        checker = []
        for node in self.t.traverse("levelorder"):
            for i in list(map(lambda x: x.dist, node.children)):
                graph.append([node.dist, i])        
        for node in self.t.traverse("levelorder"):
            for i in list(map(lambda x: x.dist, node.children)):
                checker.append([node.dist, i])        
        self.graph = graph  
        self.checker = checker
        lvl = 0
        while self.graph[lvl][0] == 0.0:
            self.graph[lvl][0] = self.names 
            #self.checker[lvl][0] = self.seq
            lvl += 1  
        check_point_dists = copy.deepcopy(self.all_dists)
        prev = ''
        
        for item in self.graph:
            if item[0] != prev:
                self.all_dists = copy.deepcopy(check_point_dists)
            else:
                for i in self.all_dists:
                    pass
            self.names = item[0]
            dist = float(item[1])
            seq = ''
            names = []
            less, more, less_index, more_index = 100, 100, [], [] #0, 0      
            
            #################################################################################
            # Generating sequence
            for al_num in range(len(self.names)):
                values = self.all_dists[self.all_id.index(self.names[al_num])]
                less, more = 100, 100
                for i in range(len(values)):
                    if float(values[i]) <= dist and abs(float(values[i])-dist) < abs(float(less)-dist):
                        less = values[i]
                    elif float(values[i]) > dist and abs(float(values[i])-dist) < abs(float(more)-dist):
                        more = values[i]
                if less != 100 and more != 100:
                    our_dist = random.choice([less, more])
                else:
                    if less != 100: our_dist = less
                    elif more != 100: our_dist = more
                #print(values)
                ind = random.choice([i for i, x in enumerate(values) if x == our_dist])
                self.all_dists[self.all_dists.index(values)][ind] = '1000'
                names.append(self.all_id[ind])
                seq += self.alignment_dict[self.all_id[ind]][self.columns[al_num]]
            i = 0
            while True:
                if i == len(graph):
                    break
                if self.graph[i][0] == dist:
                    break
                i += 1
            while True:
                if i == len(graph):
                    break
                if self.graph[i][0] != dist:
                    break
                if self.graph[i][0] == dist:
                    self.graph[i][0] = names
                    #self.checker[i][0] = seq
                i += 1
            print(seq, item[1])
            nodes.append(nd(seq))
            prev = item[0]
        self.sample = nodes
        
    # Appending information of dist
    def app_inf(self):
        j = 0
        for node in self.t.traverse("levelorder"):
            for i in list(map(lambda x:x.dist, node.children)):
                self.sample[j].dist = float(i)
                j += 1
        for node in self.sample:
            anchor = node.dist
            node.children = []
            for pair in self.checker:
                if anchor == pair[0]:
                    node.children.append(pair[1])
        for node in self.sample:
            for i in range(len(node.children)):
                for child in self.sample:
                    if node.children[i] == child.dist:
                        node.children[i] = child.seq
        j = 0
        for item in self.sample:
            item.name = ''
        for node in self.t.iter_descendants("levelorder"):
            for item in self.sample:
                if item.dist == node.dist:
                    item.name = node.name

In [184]:
f = parser(150, t) 
f.random_aln()
f.file
f.root_seq()
f.seq
f.dists()
#f.normal()
f.seq
f.make_sample()
f.app_inf()

HSHASDETS-LVHGE-TYFYEPCESN-HASTCAYILHTHYFALTGKYGPKEVGRMGIKLLSSFFRSVGL-SAAPVSHEGRGRKPALHANFGLTTK-RPSRPLGLMPAFEAPLSILLFFGYMLRSPAQSLETPVYRHGFILMRGAGRHYMY 0.555
DCATP-PTSHIFRVVASVFYKSCSAGMTPASCAYIEGEHYWKIRGRWGPDEVHNL-VDLSSGFFISLALVYAGPIVHSERSRKHVDEAVFGLPSGSLPRRPSTRLLEFLEPLRILEFNGFL-RSPFYSIEYSVYEAGLIIFHGTGPT-LY 1.001
HHMKT-EKAEVFHESMNMFYAGCDGQ-SPSICAYVENGHYFEWGGKYPPLDLLSLAIDLGDQFFETVALIDAGPLVDNTRAHLEGK-AAMGLDVSSYPSRPMGHLPAFFEPLSLLPFLGYILRPSVYSIELDLYTAPWVIFRGAGNALLY 1.244
HSFASHEGSHLIRPEMTYFYEACEVN-HACSCAYILSTEYFALTGKYGPKEVGRLG-GLSRAFFRSLALIDAAPVAHHRRGKKH-VIALLGLSTKSRPFRPCGTMPAFQAPLKILLLVGLSYRSS-QSLEEPFYSDNFILFNAAGRLYLY 0.333
HPFAA-DKAEVFRGDNGFLYAACGQKDTASSCTYIYKDHYFALGGKYPPIGLHTLVVNLGKQFFGTFALYEAEPISH-SRTHHHAH-AMFGLDTKSAPQHPLGNMPEFYNPLSVLPFLGFNLRPSIFSLEYSLYESEYYIYHGAGRTQMY 0.687
HTSAA-STS-IFRKAQDVAYEACD-K-GPTTCTYVETSEYFAAVGKHGPLEV-RLAIGLSHDFFSGVALLAAGPVMG-NRYFVHGH-ALFGLDTDSRPFRPVGH-PAFEAPLRVLLFFGYMLRDSLLSLE-PIYEIGYVVFRTAGRH-MY 0.601
DKITASPSA-LIREDKSFFYNSCHYAKSAAVCAYIETDHFFELGGKYGPHGMDRLAVV

In [185]:
with open('../../out.fasta', 'w') as out:
    for item in f.sample:
        if item.name != '':
            out.write('>'+str(item.name)+'\n'+str(item.seq)+'\n')